In [331]:
using Plots
using LinearAlgebra
plotly()

Plots.PlotlyBackend()

In [332]:
function approximate_gradient(f, x, h=1e-8)
    n = length(x)
    grad = zeros(n)
    
    for i in 1:n
        x_plus_h = copy(x)
        x_plus_h[i] += h
        
        x_minus_h = copy(x)
        x_minus_h[i] -= h
        
        grad[i] = (f(x_plus_h) - f(x_minus_h)) / (2*h)
    end
    
    return grad
end

function golden_section_search(f, a, b, tol=1e-6, max_iter=100)
    golden_ratio = (sqrt(5) + 1) / 2
    c = b - (b - a) / golden_ratio
    d = a + (b - a) / golden_ratio
    
    fc = f(c)
    fd = f(d)
    
    iter = 0
    while abs(b - a) > tol && iter < max_iter
        iter += 1
        
        if fc < fd
            b = d
            d = c
            fd = fc
            c = b - (b - a) / golden_ratio
            fc = f(c)
        else
            a = c
            c = d
            fc = fd
            d = a + (b - a) / golden_ratio
            fd = f(d)
        end
    end
    
    return (a + b) / 2
end

function swann_method(f, x0, h=0.1)
    first = x0
    second = x0 + h
    if f(second) > f(first)
        h = -h
        first, second = second, second + h
    end
    last = second + h
    
    while f(last) < f(second)
        h *= 2
        first, second, last = second, last, last + h
    end
    if second > last
        first, second, last = last, second, first
    end

    return first, last
end

swann_method (generic function with 2 methods)

In [333]:
function pirson_1(f,x_start, eps=1e-6)
    x = x_start
    trajectory = [x]
    gradient = approximate_gradient(f,x)
    H = [1 0; 0 1]
    prev_x = x
    prev_grad = gradient
    # alpha = 0.01
    while norm(gradient) > eps
        s = H * gradient
        l, r = swann_method(alpha -> f(x - alpha * s), 1e-2)
        alpha = golden_section_search(alpha -> f(x - alpha * s),  l,  r)
        x_new = [x[1] - alpha * s[1], x[2] - alpha * s[2]]
        prev_x = x
        x = x_new
        push!(trajectory, x)
        
        prev_grad = gradient
        gradient = approximate_gradient(f,x)
        d_x = [x[1] - prev_x[1], x[2] - prev_x[2]]
        delta_g = [gradient[1] - prev_grad[1], gradient[2] - prev_grad[2]]

        N = (d_x - H * delta_g) * transpose(d_x) / dot(d_x, delta_g)
        H = H + N
        println(H)
    end
    
    return x,trajectory
end

pirson_1 (generic function with 2 methods)

In [334]:
function newtone_rafson(f,x_start, eps=1e-6)
    x = x_start
    trajectory = [x]
    gradient = approximate_gradient(f,x)
    H = [1 0; 0 1]
    prev_x = x
    prev_grad = gradient
    
    while norm(gradient) > eps
        s = H * gradient
        l, r = swann_method(alpha -> f(x - alpha * s), 1e-2)
        alpha = golden_section_search(alpha -> f(x - alpha * s),  l,  r)
        x_new = [x[1] - alpha * s[1], x[2] - alpha * s[2]]
        prev_x = x
        x = x_new
        push!(trajectory, x)
        
        prev_grad = gradient
        gradient = approximate_gradient(f,x)
        d_x = [x[1] - prev_x[1], x[2] - prev_x[2]]
        delta_g = [gradient[1] - prev_grad[1], gradient[2] - prev_grad[2]]
        N = (H * delta_g) * transpose((H * delta_g)) / dot(transpose(delta_g), H * delta_g)
        H = H - N
        println(H)
    end
    
    return x,trajectory
end
     

newtone_rafson (generic function with 3 methods)

In [335]:
function greenshtadt(f,x_start, eps=1e-6)
    x = x_start
    trajectory = [x]
    gradient = approximate_gradient(f, x)
    H = [1 0; 0 1]
    prev_x = x
    prev_grad = gradient
    it = 0
    while norm(gradient) > eps
        it += 1
        s = H * gradient
        l, r = swann_method(alpha -> f(x - alpha * s), 1e-2)
        alpha = golden_section_search(alpha -> f(x - alpha * s),  l,  r)
        x_new = [x[1] - alpha * s[1], x[2] - alpha * s[2]]
        prev_x = x
        x = x_new
        push!(trajectory, x)
        
        prev_grad = gradient
        gradient = approximate_gradient(f, x)
        d_x = [x[1] - prev_x[1], x[2] - prev_x[2]]
        delta_g = [gradient[1] - prev_grad[1], gradient[2] - prev_grad[2]]
        
        multiplier = 1 / dot(transpose(delta_g), H * delta_g)
        first_add = d_x*transpose(delta_g)*H+H*delta_g*transpose(d_x)
        second_chisl_multyp = transpose(delta_g)*d_x - transpose(delta_g)*H*delta_g
        second_chisl_multyp_2 = H*delta_g*transpose(delta_g)*H
        second_chisl_multyp = second_chisl_multyp*second_chisl_multyp_2
        znam_second = dot(transpose(delta_g),H*delta_g)
        second_add = second_chisl_multyp/znam_second
        first_add = first_add-second_add
        N = first_add * multiplier
        H = H - N
        println(H)
    end
    
    return x,trajectory
end

greenshtadt (generic function with 2 methods)

In [336]:
function goldfarb(f,x_start, eps=1e-6)
    x = x_start
    trajectory = [x]
    gradient = approximate_gradient(f,x)
    H = [1 0; 0 1]
    prev_x = x
    prev_grad = gradient
    
    it = 0
    while norm(gradient) > eps
        s = H * gradient
        l, r = swann_method(alpha -> f(x - alpha * s), 1e-2)
        alpha = golden_section_search(alpha -> f(x - alpha * s),  l,  r)
        x_new = [x[1] - alpha * s[1], x[2] - alpha * s[2]]
        prev_x = x
        x = x_new
        push!(trajectory, x)
        
        prev_grad = gradient
        gradient = approximate_gradient(f,x)
        d_x = [x[1] - prev_x[1], x[2] - prev_x[2]]
        delta_g = [gradient[1] - prev_grad[1], gradient[2] - prev_grad[2]]
        
        multiplier = 1 / dot(transpose(delta_g), H * delta_g)
        
        first = d_x*transpose(delta_g)*H + H*delta_g*transpose(d_x)
        
        ss = transpose(delta_g)*d_x
        sss = H * delta_g * transpose(delta_g) * H
        second = 1 + ss/dot(transpose(delta_g), H*delta_g)
        second = second * sss 
        N = multiplier * (first - second)
        H = H + N
        println(H)
    end
    
    return x,trajectory
end

goldfarb (generic function with 2 methods)

In [337]:
using Plots
using LinearAlgebra

function fletcher(f,x_start, eps=1e-6)
    x = x_start
    trajectory = [x]
    gradient = approximate_gradient(f,x)
    H = [1 0; 0 1]
    prev_x = x
    prev_grad = gradient
    
    while norm(gradient) > eps
        s = H * gradient
        l, r = swann_method(alpha -> f(x - alpha * s), 1e-2)
        alpha = golden_section_search(alpha -> f(x - alpha * s),  l,  r)
        x_new = [x[1] - alpha * s[1], x[2] - alpha * s[2]]
        prev_x = x
        x = x_new
        push!(trajectory, x)
        
        prev_grad = gradient
        gradient = approximate_gradient(f,x)
        d_x = [x[1] - prev_x[1], x[2] - prev_x[2]]
        delta_g = [gradient[1] - prev_grad[1], gradient[2] - prev_grad[2]]
        
        first_mnog = 1 / dot(transpose(delta_g), H*delta_g)
        first_add = d_x*transpose(delta_g)*H + H*delta_g*transpose(d_x)
        second_multy_chisl = transpose(delta_g) * d_x
        second_multy_znam = dot(transpose(delta_g), H*delta_g)
        second_multy_chisl = second_multy_chisl/second_multy_znam
        sec_2 = 1 + second_multy_chisl
        sec_3 = H * delta_g * transpose(delta_g)*H
        second_add = sec_2 * sec_3
        chisl=first_add-second_add
        E = [1 0; 0 1]
        first_chisl = d_x * transpose(delta_g)
        first_znam = dot(transpose(d_x), delta_g)
        first_mnog = first_chisl / first_znam
        first_mnog = E - first_mnog
        
        second_chisl = delta_g * transpose(d_x)
        second_znam = dot(transpose(d_x), delta_g)
        second_mnog = second_chisl / second_znam
        second_mnog = E - second_mnog
        
        N = first_mnog * H * second_mnog
        chisl = d_x * transpose(d_x)
        znam = dot(transpose(d_x), delta_g)
        H = N + chisl / znam

        println(H)
    end
    
    return x,trajectory
end

fletcher (generic function with 2 methods)

In [338]:
function plot_optimization_paths(f, x_range, y_range, paths_with_names, title="Сравнение методов оптимизации", global_min=nothing)
    z = [f([x, y]) for y in y_range, x in x_range]
    
    clamp_level = maximum(filter(isfinite, z)) / 2 
    z_clamped = [min(val, clamp_level) for val in z]
    
    p = contour(x_range, y_range, z_clamped,
                fill=false,
                levels=20,
                color=:thermal,
                xlabel="x",
                ylabel="y",
                title=title,
                size=(800, 600))
    
    colors = [:red, :green, :blue, :purple, :orange]
    for (i, (name, path)) in enumerate(paths_with_names)
        x_coords = [point[1] for point in path]
        y_coords = [point[2] for point in path]
        
        plot!(p, x_coords, y_coords, 
              label=name, 
              line=(colors[i], 2),
              marker=(:circle, 2, 0.5))
        
        annotate!(p, x_coords[1], y_coords[1], text("Старт", :left, 8, :white))
        annotate!(p, x_coords[end], y_coords[end], text("Финиш", :right, 8, :white))
    end
    
    if global_min !== nothing
        scatter!(p, [global_min[1]], [global_min[2]], 
                label="Глобальный минимум", 
                color=:white, 
                markersize=5,
                markerstrokewidth=1,
                markerstrokecolor=:black)
    end
    
    return p
end

plot_optimization_paths (generic function with 3 methods)

In [339]:
# 1. Функция Розенброка
function rosenbrock(x)
    return (1.0 - x[1])^2 + 100.0*(x[2] - x[1]^2)^2
end

# 2. Функция Растригина
function rastrigin(x)
    return 20 + x[1]^2 - 10*cos(2*pi*x[1]) + x[2]^2 - 10*cos(2*pi*x[2])
end

# 3. Функция Швефеля
function schwefel(x)
    return 418.9829*2 - (x[1]*sin(sqrt(abs(x[1]))) + x[2]*sin(sqrt(abs(x[2]))))
end

function my(x)
    return (x[1] - 4*x[2])^2 + (x[2] + 5)^2
end

a = 7.0
b = 5.0
function quadratic(x)
    return a * x[1]^2 + b * x[2]^2
end


function run_optimization(f, x0, title, x_range, y_range, global_min=nothing)
    println("\n======= $(title) =======")

    result_pirson_1, path_pirson_1 = pirson_1(f, x0)
    
    println("Метод Пирсона (1 модификация): ", result_pirson_1)
    println("Значение функции: ", f(result_pirson_1))
    println("Количество итераций: ", length(path_pirson_1))
    println()
    
    result_newtone_rafson, path_newtone_rafson = newtone_rafson(f, x0)
    
    println("Метод Ньютона-Рафсона: ", result_newtone_rafson)
    println("Значение функции: ", f(result_newtone_rafson))
    println("Количество итераций: ", length(path_newtone_rafson))
    println()


    result_greenshtadt, path_greenshtadt= greenshtadt(f, x0)
    println("Метод Гринштадта: ", result_greenshtadt)
    println("Значение функции: ", f(result_greenshtadt))
    println("Количество итераций: ", length(path_greenshtadt))
    println()

    result_goldfarb, path_goldfarb= goldfarb(f, x0)
    println("Метод Гольдфарба: ", result_goldfarb)
    println("Значение функции: ", f(result_goldfarb))
    println("Количество итераций: ", length(path_goldfarb))
    println()

    result_fletcher, path_fletcher= fletcher(f, x0)
    println("Метод Флэтчера: ", result_fletcher)
    println("Значение функции: ", f(result_fletcher))
    println("Количество итераций: ", length(path_fletcher))
    println()
    
    paths = [
        ("Пирсон - 1",path_pirson_1),
        ("Ньютона-Рафсон", path_newtone_rafson),
        ("Гринштадт",path_greenshtadt),
        ("Гольдфарб",path_goldfarb),
        ("Флэтчер",path_fletcher),
    ]
    
    p = plot_optimization_paths(f, x_range, y_range, paths, title, global_min)
    
    display(p)
    
    return nothing
end

run_optimization (generic function with 2 methods)

In [340]:
# Для функции Розенброка
x0_rosenbrock = [-1.2, 2.0]
x_range_rosenbrock = -3.0:0.1:3.0
y_range_rosenbrock = -1.0:0.1:4.0
global_min_rosenbrock = [1.0, 1.0]
run_optimization(rosenbrock, x0_rosenbrock, "Функция Розенброка", x_range_rosenbrock, y_range_rosenbrock, global_min_rosenbrock)


======= Функция Розенброка =======
[0.14185240124532017 -0.36351184208626935; -0.3617431903247516 0.8467653656924047]
[-0.004841478590745546 -0.018358844472713576; -0.0010621411911448941 -0.0018736193149287983]
[-0.004666846322931016 -0.01833317507229524; -0.00043359293647618957 -0.0017812282749837974]
[-0.004673340347820481 -0.01833380709641354; -0.00045453730593684745 -0.0017832666636359918]
[-0.00467075875018453 -0.01833355599295213; -0.0004543092919294016 -0.0017832444854678211]
[-0.004650915080285863 -0.01833162586478923; -0.0004523791628263022 -0.001783056748191453]
[-0.004664648270377209 -0.01833296164678405; -0.00045371494490987357 -0.0017831866752982919]
[-0.0047327760704284575 -0.01833958821070799; -0.0004603415227119976 -0.0017838312204233315]
[-0.004624180580714992 -0.018329025489558972; -0.0004497787978175021 -0.0017828038193965603]
[-0.004669661161566927 -0.018333449233293976; -0.0004542025499741843 -0.0017832341029488316]
[-0.0046910856951530855 -0.01833553312684103; -0

Excessive output truncated after 524306 bytes.

 -0.0017833173824206917]
[-0.00463726242895544 -0.018330298022233377; -0.0004510512259029413 -0.0017829275944378068]
[-0.004650914992286382 -0.018331625961541118; -0.00045237916572120297 -0.0017830567587156234]
[-0.004669661191453075 -0.018333449341532432; -0.00045420255288091565 -0.0017832341134761205]
[-0.004664648299996598 -0.018332961754154335; -0.00045371494779065816 -0.0017831866857411357]
[-0.004718797295541146 -0.018338228647920812; -0.0004589818418799023 -0.0017836989791155256]
[-0.004637912007336262 -0.018330361205666806; -0.0004511144084082499 -0.0017829337400974154]
[-0.004650914992600243 -0.018331625962682136; -0.0004523791657517311 -0.0017830567588266058]
[-0.004673168932642277 -0.018333790528715966; -0.0004545437422274723 -0.0017832672999131803]
[-0.004678462833128758 -0.018334305448913325; -0.00045505864184339386 -0.0017833173824964338]
[-0.004718797295769068 -0.018338228648738516; -0.0004589818419020712 -0.0017836989791950595]
[-0.004650914992825088 -0.0183316259634995

LoadError: InterruptException: